### Run notebook 01_LoL_predictor_EDA_preparation before running this one, as it prepares the data!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

In [ ]:
import wandb
wandb.login()

In [22]:
pd.set_option('display.max_columns', 500)
PREPROCESSED_PATH = '../data/processed_high_diamond.csv'

data = pd.read_csv(PREPROCESSED_PATH)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
data.drop(columns={"gameId"}, inplace=True)

In [4]:
# split data to train and test by 70/30 ratio
train, test = train_test_split(data, test_size=0.3)

# further split test part to 50/50 with validation - final split will be 70/15/15
test, validation = train_test_split(test, test_size=0.5)

In [5]:
train.shape

(6535, 17)

In [6]:
test.shape

(1401, 17)

In [7]:
validation.shape

(1401, 17)

In [8]:
trainX = train.drop(columns={"blueWins"})
trainy = train.blueWins

In [9]:
testX = test.drop(columns={"blueWins"})
testy = test.blueWins

In [10]:
model = nn.Sequential(
    nn.Linear(16, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
    # Softmax
)

In [11]:
trainX = torch.tensor(trainX.values)
testX = torch.tensor(testX.values)

In [12]:
trainy = torch.tensor(trainy.values)
testy = torch.tensor(testy.values)

# https://stackoverflow.com/a/60440460/12342419
trainy = trainy.type(torch.LongTensor)
testy = testy.type(torch.LongTensor)

In [13]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

In [14]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
trainX = trainX.to(torch.float32).to(device)
testX = testX.to(torch.float32).to(device)
trainy = trainy.to(torch.float32).to(device)
testy = testy.to(torch.float32).to(device)

In [17]:
trainy = trainy.reshape((trainy.shape[0], 1))

In [18]:
model

Sequential(
  (0): Linear(in_features=16, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Sigmoid()
)

In [19]:
model.to(device)
print(device)

cuda:0


In [20]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1050'

In [24]:
num_epochs = 10000
for n in range(num_epochs):
    model.train()
    y_pred = model(trainX)

    if n % 100 == 0:
        print(n)
        #print(y_pred)
        print(accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred)))

    loss = compute_loss(y_pred, trainy)
    optimizer.zero_grad()
    loss.backward()
    #loss.backward(retain_graph=False)
    optimizer.step()
print(loss)

0
49.288446824789595
100
72.74674827850038
200
73.25172149961745
300
73.60367253251722
400
73.78729915837796
500
74.18515684774293
600
74.5983167559296
700
74.96557000765111
800
75.2257077276205
900
75.91430757459831
1000
75.97551644988523
1100
76.48048967100229
1200
77.00076511094109
1300
76.8018362662586
1400
76.44988523335884
1500
77.49043611323641
1600
77.64345830145372
1700
77.19969395562356
1800
78.2096403978577
1900
78.2096403978577
2000
78.57689364957919
2100
79.03596021423105
2200
78.51568477429227
2300
78.9441469013007
2400
79.64804896710022
2500
78.83703136954858
2600
79.41851568477429
2700
79.18898240244836
2800
80.0
2900
80.59678653404744
3000
80.56618209640398
3100
79.3726090283091
3200
80.76511094108646
3300
80.78041315990818
3400
81.10175975516451
3500
81.22417750573834
3600
80.2142310635042
3700
81.6832440703902
3800
81.5761285386381
3900
81.75975516449884
4000
81.82096403978576
4100
81.95868400918133
4200
81.91277735271615
4300
82.09640397857689
4400
82.28003060443766

In [ ]:
y_pred.shape

torch.Size([6535, 1])

In [ ]:
torch.round(y_pred[:10])

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]], grad_fn=<RoundBackward0>)

In [ ]:
testy[:10]

tensor([0., 0., 1., 0., 1., 1., 1., 0., 0., 0.])

In [ ]:
def predict(model, test_loader):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_preds, all_preds_raw, all_labels

In [ ]:
testX[0]

tensor([0.4339, 0.3935, 0.4414, 0.5694, 0.5528, 0.7431, 0.5000, 0.8571, 0.5714,
        0.3333, 0.1111, 0.2778, 0.0000, 0.5040, 0.0000, 0.5440])

In [ ]:
testy

tensor([0., 0., 1.,  ..., 0., 1., 1.])

In [ ]:
len(model(testX))

1401

In [27]:
lol = model(testX[20:30])
print(lol)
#print(lol.flatten())
print(testy[20:30])

tensor([[0.9970],
        [0.1816],
        [0.5450],
        [0.0798],
        [0.1047],
        [0.9925],
        [0.3970],
        [0.0391],
        [0.4638],
        [0.0498]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([0., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')


In [ ]:
testy[:10]

tensor([0., 0., 1., 0., 1., 1., 1., 0., 0., 0.])

In [ ]:
predict(model, test)

AttributeError: 'str' object has no attribute 'x'

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

In [ ]:
import tensorflow as tf